# Test: RNA-seq Data from GEO via ARCHS4

To test performance on some larger and messier RNA-seq data, a larger gene expression matrix alongside more cancer type labels has been retrieved from the Gene Expression Omnibus via the ARCHS4 platform.

---

#### Imports

In [1]:
from time import perf_counter

import numpy as np
import pandas as pd

import seaborn as sns
sns.set_theme()

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.decomposition import PCA
from sklearn.random_projection import SparseRandomProjection, GaussianRandomProjection

# For comparison
from sklearn.svm import SVC, LinearSVC

# Local import
from svm import MultiGDSVM
import dim_reduction

---

#### Load Data

In [2]:
path = "data/"

In [3]:
X = pd.read_csv(path + "cancer_expression_matrix.tsv", sep="\t")
X

,Unnamed: 0,GSM1228197,GSM1177221,GSM988637,GSM1228191,GSM1097789,GSM1177220,GSM1154037,GSM862357,GSM1113310,...,GSM5575411,GSM5575413,GSM5575418,GSM5575420,GSM5575421,GSM5575423,GSM5575426,GSM5575431,GSM5575435,GSM5575439
0,A1BG,30,179,110,100,998,155,359,5,139,...,77,44,24,14,39,49,48,135,24,28
1,A1CF,342,9,30,759,9,12,14,4,2,...,978,1482,1084,800,1629,357,2886,3299,1900,1167
2,A2M,3371,19413,79,13801,7191,20213,4,0,1206,...,10775,20523,16561,23239,49016,24121,20164,12002,18110,11614
3,A2ML1,3,215,45,6,110,19,84,3,9,...,49,12,19,32,14,24,11,23,11,18
4,A2MP1,5,37,115,7,5,42,29,0,0,...,1,0,0,1,0,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35233,bP-2171C21.4,0,0,0,0,2,1,33,0,0,...,0,0,0,0,0,0,0,0,0,0
35234,bP-2171C21.5,0,10,1,0,5,17,0,0,0,...,22,0,28,63,38,31,50,10,1,3
35235,bP-2171C21.6,0,0,6,0,26,1,43,0,0,...,4,0,0,0,0,0,0,0,0,0
35236,bP-2189O9.2,1,16,72,2,22,137,557,45,7,...,1339,517,244,513,830,199,945,7038,633,294


In [4]:
y = pd.read_csv(path + "cancer_expression_labels.tsv", sep="\t")
y

,sample,label
0,GSM1228197,colorectal
1,GSM1177221,prostate
2,GSM988637,NaN
3,GSM1228191,colorectal
4,GSM1097789,NaN
...,...,...
17642,GSM5575423,colorectal
17643,GSM5575426,colorectal
17644,GSM5575431,colorectal
17645,GSM5575435,colorectal


---

#### Preprocess Data

In [5]:
X = X.rename(columns={X.columns[0]: "gene"})
X = X.set_index("gene").transpose().reset_index().rename(columns={"index": "sample"})

In [6]:
X["sample"]

0        GSM1228197
1        GSM1177221
2         GSM988637
3        GSM1228191
4        GSM1097789
            ...    
17642    GSM5575423
17643    GSM5575426
17644    GSM5575431
17645    GSM5575435
17646    GSM5575439
Name: sample, Length: 17647, dtype: object

In [7]:
X

gene,sample,A1BG,A1CF,A2M,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,AAAS,...,bP-21201H5.1,bP-21264C1.1,bP-2168N6.1,bP-2168N6.3,bP-2171C21.2,bP-2171C21.4,bP-2171C21.5,bP-2171C21.6,bP-2189O9.2,yR211F11.2
0,GSM1228197,30,342,3371,3,5,0,226,0,1289,...,3,2,0,0,0,0,0,0,1,1
1,GSM1177221,179,9,19413,215,37,3,599,0,974,...,38,3,0,0,0,0,10,0,16,2
2,GSM988637,110,30,79,45,115,3,123,0,1026,...,56,115,0,0,0,0,1,6,72,1
3,GSM1228191,100,759,13801,6,7,0,290,2,1122,...,11,3,0,0,0,0,0,0,2,0
4,GSM1097789,998,9,7191,110,5,4,874,0,7384,...,38,259,0,0,4,2,5,26,22,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17642,GSM5575423,49,357,24121,24,0,0,90,7,2184,...,1,56,0,0,10,0,31,0,199,0
17643,GSM5575426,48,2886,20164,11,2,0,125,4,2962,...,1,458,0,0,2,0,50,0,945,0
17644,GSM5575431,135,3299,12002,23,0,0,12,0,3027,...,0,1654,0,0,15,0,10,0,7038,0
17645,GSM5575435,24,1900,18110,11,1,1,131,1,2699,...,0,194,0,0,4,0,1,0,633,0


In [8]:
y["sample"]

0        GSM1228197
1        GSM1177221
2         GSM988637
3        GSM1228191
4        GSM1097789
            ...    
17642    GSM5575423
17643    GSM5575426
17644    GSM5575431
17645    GSM5575435
17646    GSM5575439
Name: sample, Length: 17647, dtype: object

In [9]:
df = X.merge(y, on="sample", how="inner")
df = df.dropna()

col = df.pop("label")
df.insert(1, col.name, col)

df

,sample,label,A1BG,A1CF,A2M,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,bP-21201H5.1,bP-21264C1.1,bP-2168N6.1,bP-2168N6.3,bP-2171C21.2,bP-2171C21.4,bP-2171C21.5,bP-2171C21.6,bP-2189O9.2,yR211F11.2
0,GSM1228197,colorectal,30,342,3371,3,5,0,226,0,...,3,2,0,0,0,0,0,0,1,1
1,GSM1177221,prostate,179,9,19413,215,37,3,599,0,...,38,3,0,0,0,0,10,0,16,2
3,GSM1228191,colorectal,100,759,13801,6,7,0,290,2,...,11,3,0,0,0,0,0,0,2,0
5,GSM1177220,prostate,155,12,20213,19,42,14,635,1,...,17,15,0,0,1,1,17,1,137,2
6,GSM1154037,prostate,359,14,4,84,29,3,39,1,...,8,581,0,0,7,33,0,43,557,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17642,GSM5575423,colorectal,49,357,24121,24,0,0,90,7,...,1,56,0,0,10,0,31,0,199,0
17643,GSM5575426,colorectal,48,2886,20164,11,2,0,125,4,...,1,458,0,0,2,0,50,0,945,0
17644,GSM5575431,colorectal,135,3299,12002,23,0,0,12,0,...,0,1654,0,0,15,0,10,0,7038,0
17645,GSM5575435,colorectal,24,1900,18110,11,1,1,131,1,...,0,194,0,0,4,0,1,0,633,0


In [10]:
df.shape[0] * df.shape[1]

548122960

In [11]:
for cancer_type in df.label.unique():
    print(cancer_type)

colorectal
prostate
breast
lung
gastric
pancreatic
ovarian
kidney


In [12]:
X = df.iloc[:, 2:]
y = df.iloc[:, 1]

In [13]:
# Scale input features
scaler = MinMaxScaler()
#scaler = StandardScaler()
X = scaler.fit_transform(X)

# Encode target labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

---

#### Data Prep

In [17]:
#num_features = 10_000

#X_dim = dim_reduction.GaussianRandProj(num_features).fit_transform(X)
#X_dim = GaussianRandomProjection(num_features).fit_transform(X)
#X_dim = SparseRandomProjection(num_features).fit_transform(X)

In [23]:
# 90:5:5 split
random_state = 64
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_state)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=random_state)

---

#### Experimentation

In [29]:
%%time
svm = MultiGDSVM(
    kernel="linear", C=0.2, beta1=0.95, beta2=0.999, batch_size=256, nystrom_rows=500,
)

t0 = perf_counter()
svm.fit(X_train, y_train)
print(f"Time: {perf_counter() - t0:.4f} s")

y_pred = svm.predict(X_test)
print(f"Test Accuracy: {(y_pred == y_test).sum() / y_pred.shape[0]:.4f}")

converged (epoch=1488)
converged (epoch=1643)
converged (epoch=1247)
converged (epoch=1026)
converged (epoch=1398)
converged (epoch=1568)
converged (epoch=1135)
converged (epoch=1502)
Time: 137.0813 s
Test Accuracy: 0.9614
CPU times: total: 5min 3s
Wall time: 2min 17s


In [30]:
%%time
svc = LinearSVC(max_iter=10_000, C=1.0)

t0 = perf_counter()
svc.fit(X_train, y_train)
print(f"Time: {perf_counter() - t0:.4f} s")

y_pred = svc.predict(X_test)
print(f"Test Accuracy: {(y_pred == y_test).sum() / y_pred.shape[0]:.4f}")

Time: 205.4961 s
Test Accuracy: 0.9820
CPU times: total: 14.6 s
Wall time: 3min 25s


*Validation*

In [31]:
print("custom")
y_pred = svm.predict(X_val)
print(f"Val Accuracy: {(y_pred == y_val).sum() / y_pred.shape[0]:.4f}")

print("sklearn")
y_pred = svc.predict(X_val)
print(f"Val Accuracy: {(y_pred == y_val).sum() / y_pred.shape[0]:.4f}")

custom
Val Accuracy: 0.9550
sklearn
Val Accuracy: 0.9781


---

#### Grid search

*Iter 1*

In [ ]:
grid = {
    "C": [0.1, 0.2, 0.5, 1.0],
    "batch_size": [64, 128],
}

svm = MultiGDSVM(kernel="linear", beta1=0.95, beta2=0.9999, verbose=False)
clf = GridSearchCV(svm, grid, scoring="accuracy")

clf.fit(X_train, y_train)

GridSearchCV(estimator=<svm.MultiGDSVM object at 0x0000023791409420>,
             param_grid={'C': [0.1, 0.2, 0.5, 1.0], 'batch_size': [64, 128]},
             scoring='accuracy')

In [ ]:
svm = clf.best_estimator_

print(svm.get_params())

{'C': 0.1, 'l1_reg': 0.0, 'kernel': 'linear', 'degree': 3, 'gamma': 1, 'batch_size': 128, 'nystrom_rows': 200, 'n_dims': 0, 'dim_method': 'pca', 'iters': 10000, 'eps': 1e-10, 'lr': 0.01, 'beta1': 0.95, 'beta2': 0.9999, 'shrink_thresh': 1e-06, 'verbose': False}


In [ ]:
t0 = perf_counter()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(f"Time: {perf_counter() - t0}")

print(f"Accuracy: {(y_pred == y_test).sum() / y_pred.shape[0]:.4f}")

Time: 41.742889899993315
Accuracy: 0.9087


*Iter 2*

In [ ]:
grid = {
    "nystrom_rows": [200, 500, 1000],
}

svm = MultiGDSVM(kernel="linear", C=0.1, batch_size=128, beta1=0.95, beta2=0.9999, verbose=False)
clf = GridSearchCV(svm, grid, scoring="accuracy")

clf.fit(X_train, y_train)

GridSearchCV(estimator=<svm.MultiGDSVM object at 0x0000023791392A10>,
             param_grid={'nystrom_rows': [200, 500, 1000]}, scoring='accuracy')

In [ ]:
svm = clf.best_estimator_

print(svm.get_params())

{'C': 0.1, 'l1_reg': 0.0, 'kernel': 'linear', 'degree': 3, 'gamma': 1, 'batch_size': 128, 'nystrom_rows': 1000, 'n_dims': 0, 'dim_method': 'pca', 'iters': 10000, 'eps': 1e-10, 'lr': 0.01, 'beta1': 0.95, 'beta2': 0.9999, 'shrink_thresh': 1e-06, 'verbose': False}


In [ ]:
t0 = perf_counter()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(f"Time: {perf_counter() - t0}")

print(f"Accuracy: {(y_pred == y_test).sum() / y_pred.shape[0]:.4f}")

Time: 55.18591539998306
Accuracy: 0.9643


---

#### Other Experiment with Scikit-learn

In [25]:
%%time
svc = LinearSVC(max_iter=10_000, C=3.0)

t0 = perf_counter()
svc.fit(X_train, y_train)
print(f"Time: {perf_counter() - t0:.4f} s")

y_pred = svc.predict(X_test)
print(f"Test Accuracy: {(y_pred == y_test).sum() / y_pred.shape[0]:.4f}")

Time: 205.1118 s
Test Accuracy: 0.9820
CPU times: total: 20.2 s
Wall time: 3min 25s
